In [51]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm, trange
import torch
import pickle
import io
import re

In [ ]:
pip install --upgrade scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives


In [ ]:
%cd /content/drive/Shareddrives
%cd 'STATS 209 '
%ls
%cd 'stats 209 rawr'
%ls

/content/drive/Shareddrives
/content/drive/Shareddrives/STATS 209 
'stats 209 rawr'/
/content/drive/Shareddrives/STATS 209 /stats 209 rawr
 11_26_23_Second_Degree_Model_2_Data.csv                   EM_Equation3.png
 12_9_23_Equation_1_biogridmouse.csv                       equation_1_biodiseasome_batch.pkl
 12_9_23_Equation_1_RandomGraph.csv                        equation_1_biodiseasome.csv
 12_9_23_Equation_1_Simulations_Generated_Graph.ipynb      FIXED_Replication1_HuEtAl_Graph.png
 12_9_23_Equation_1_Simulations_Real_Data.ipynb            FIXED_Replication2_HuEtAl_Graph.png
 12_9_23_Equation_2_RandomGraph.csv                        FIXED_Replication3_HuEtAl_Graph.png
 12_9_23_Equation_3_RandomGraph.csv                        FIXED_Replication_HuEtAl_Graph.png
 27done12_9_23_Equation_3_RandomGraph.pkl                  grapheq1.ibynb
 ADE_Across_eqs.png                                        grapheq2.ibynb
 AIE_1_Across_eqs.png                                      grapheq3.ibynb
 AIE

In [52]:
def remove_non_printable_chars(file_path, output_file_path):
    # Regular expression to match non-printable characters
    # This matches any character that is not a space, tab, newline, or printable ASCII character
    non_printable_pattern = re.compile('[^ \t\n\r\f\v\x20-\x7E]')

    with open(file_path, 'r') as file:
        content = file.read()

    # Remove non-printable characters
    cleaned_content = non_printable_pattern.sub('', content)

    with open(output_file_path, 'w') as file:
        file.write(cleaned_content)

    print(f"Cleaned file written to {output_file_path}")

%cd /content/drive/Shareddrives
%cd 'STATS 209 '
%cd 'stats 209 rawr'
# Replace with your file paths
file_path = 'bio-yeast.mtx'
output_file_path = 'bio-yeast-cleaned.mtx'

remove_non_printable_chars(file_path, output_file_path)

/content/drive/Shareddrives
/content/drive/Shareddrives/STATS 209 
/content/drive/Shareddrives/STATS 209 /stats 209 rawr
Cleaned file written to bio-yeast-cleaned.mtx


In [75]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix

#fh = io.BytesIO()

def mtx_to_adjacency_matrix(file_path):
    # Read the Matrix Market file
    matrix = mmread(file_path)

    # Convert the matrix to a compressed sparse row (CSR) format
    matrix_csr = csr_matrix(matrix)

    # Convert the CSR matrix to a dense array (adjacency matrix)
    adjacency_matrix = matrix_csr.toarray()

    return adjacency_matrix

# Example usage

num_lines_to_display = 10
%cd /content/drive/Shareddrives
%cd 'STATS 209 '
%cd 'stats 209 rawr'
########
corrected_file_path = 'bio-celegans-cleaned.mtx'
 # Path for the corrected file
file_path = 'bio-celegans.mtx'

# Read the existing file content
with open(file_path, 'r') as file:
    lines = file.readlines()

# Check if the header needs correction
if lines[0].startswith('%MatrixMarket'):
    lines[0] = '%' + lines[0]

# Write the corrected content to a new file
with open(corrected_file_path, 'w') as file:
    file.writelines(lines)

#########

# Read and display the first few lines of the file
with open(corrected_file_path, 'r') as file:
    for _ in range(num_lines_to_display):
        line = file.readline()
        print(line.strip())



%cd /content/drive/Shareddrives
%cd 'STATS 209 '
%cd 'stats 209 rawr'
A = mtx_to_adjacency_matrix('bio-celegans-cleaned.mtx')
graph = nx.Graph(a)

#print("Adjacency Matrix:")
#print(adjacency_matrix)

/content/drive/Shareddrives
/content/drive/Shareddrives/STATS 209 
/content/drive/Shareddrives/STATS 209 /stats 209 rawr
%%MatrixMarket matrix coordinate pattern symmetric
453 453 2025
186 1
207 1
218 1
227 1
228 1
229 1
230 1
3 2
/content/drive/Shareddrives
/content/drive/Shareddrives/STATS 209 
/content/drive/Shareddrives/STATS 209 /stats 209 rawr


In [84]:
edges_df = pd.read_csv('bio-celegans-corrected.mtx', header=None, delimiter=' ', names=['Node1', 'Node2'])

# Create a graph from the edges DataFrame
G = nx.from_pandas_edgelist(edges_df, 'Node1', 'Node2')

# Create an adjacency matrix
A = nx.adjacency_matrix(G).todense()
vertices = len(A[0])


In [86]:
device = torch.device("cuda")
vertices = len(A[0])

In [98]:
#second degree neighbor matrix
A_gpu = torch.tensor(A).float().to(device)
A_2_gpu = torch.matmul(A_gpu, A_gpu)
A_2 = A_2_gpu.cpu().numpy()
A_2 = np.where(A_2 > 0, 1, 0)
np.fill_diagonal(A_2, 0)
A_2 = A_2 - A # second degree neighbors
A_2_gpu = torch.tensor(A_2).float().to(device)
print(A_2.shape)

(2029, 2029)


In [78]:
#set parameters
#vertices = 1000
#adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000

In [79]:
replicate = {}

In [89]:
#beta 1 is treatment assignment
#beta 2 is neighbors
#beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []




In [103]:
#beta 1 is treatment assignment
#beta 2 is neighbors
#beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  print(f'\nBETAS {beta_1}, {beta_2}, {beta_3}\n')
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []
    for i in trange(1000):
      # treatment assignment
      W = np.random.binomial(1, pi, vertices)
      W_gpu = torch.tensor(W).float().to(device)

      # first deg effect
      if beta_2 != 0:
        W_A = torch.matmul(W_gpu, A_gpu).cpu().numpy()
      else:
        W_A = np.zeros(vertices)

      # second deg effect
      if beta_3 != 0:
        W_A_2 = torch.matmul(W_gpu, A_2_gpu).cpu().numpy()
      else:
        W_A_2 = np.zeros(vertices)

      # noise
      noise = np.random.normal(0, 1, vertices)

      # generate Y
      s1 = np.where(np.sum(A, axis=1) == 0, 0.000001, np.sum(A, axis=1)) # circumvent div by 0 errors
      # A_1_frac = np.where(np.sum(A, axis=1) != 0, np.divide(W_A, np.sum(A, axis=1, keepdims=True)), 0)
      A_1_frac = np.where(W_A == 0, 0, np.divide(W_A, s1))
      #print("YUHHH", A_2.shape)
      s2 = np.where(np.sum(A_2, axis=1) == 0, 0.000001, np.sum(A_2, axis=1)) # circumvent div by 0 errors
      # A_2_frac = np.where(np.logical_and(np.sum(A_2, axis=1) != 0, W_A_2 != 0), np.divide(W_A_2, np.sum(A_2, axis=1, keepdims=True)), 0)

      #print(W_A_2.shape, A_2.shape)
      #print(W_A.shape, s1.shape)

      A_2_frac = np.where(W_A_2 == 0, 0, np.divide(W_A_2, s2)) # W_A_2 == 0 --> A_2 == 0 (there are more W_A_2 = 0s)
      Y = A_2_frac * beta_3 + A_1_frac * beta_2 + W * (beta_1) + noise

      # ADE
      average_direct_effect = ((np.dot(Y,W))/(pi) - (np.dot(Y, (1 - W)))/(1-pi)) / vertices
      ADE.append(average_direct_effect)

      # AIE
      # E = first degree
      average_indirect_effect1 = np.dot(Y, ((W_A) / pi - ((1 - W) @ A) / (1 - pi))) / vertices
      average_indirect_effect2 = np.dot(Y, ((W_A_2) / pi - ((1 - W) @ A_2) / (1 - pi))) / vertices
      AIE_1.append(average_indirect_effect1)
      AIE_2.append(average_indirect_effect2)

    str1 = str(pi)+" ADE_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str2 = str(pi)+" AIE_1_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str3 = str(pi)+" AIE_2_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    replicate[str1] = ADE
    replicate[str2] = AIE_1
    replicate[str3] = AIE_2

    with open('equation_1_biodiseasome_batch.pkl', 'wb') as fp:
      pickle.dump(replicate, fp)

In [81]:
betas = []
# baseline where no interference
betas.append((1, 0, 0))

#only first degree neighbors
betas.append((1/3, 2/3, 0))
betas.append((0.5, 0.5, 0))
betas.append((2/3, 1/3, 0))

#only second degree neighbors
betas.append((1/3, 0, 2/3))
betas.append((0.5, 0, 0.5))
betas.append((2/3, 0, 1/3))

#first and second degree neighbors
betas.append((1/3, 5/9, 1/9))
betas.append((1/3, 1/3, 1/3))
betas.append((1/3, 1/9, 5/9))

In [104]:
for beta in betas:
  add_test_to_dictionary(beta[0], beta[1], beta[2], replicate)


BETAS 1, 0, 0



100%|██████████| 1000/1000 [01:03<00:00, 15.63it/s]



BETAS 0.3333333333333333, 0.6666666666666666, 0



100%|██████████| 1000/1000 [01:05<00:00, 15.20it/s]



BETAS 0.5, 0.5, 0



...

100%|██████████| 1000/1000 [01:11<00:00, 13.94it/s]



BETAS 0.3333333333333333, 0.3333333333333333, 0.3333333333333333



100%|██████████| 1000/1000 [01:11<00:00, 13.92it/s]



BETAS 0.3333333333333333, 0.1111111111111111, 0.5555555555555556



100%|██████████| 1000/1000 [01:10<00:00, 14.23it/s]


In [105]:
print(replicate.keys())

dict_keys(['0.25 ADE_1_0_0', '0.25 AIE_1_1_0_0', '0.25 AIE_2_1_0_0', '0.5 ADE_1_0_0', '0.5 AIE_1_1_0_0', '0.5 AIE_2_1_0_0', '0.75 ADE_1_0_0', '0.75 AIE_1_1_0_0', '0.75 AIE_2_1_0_0', '0.25 ADE_0.33_0.67_0', '0.25 AIE_1_0.33_0.67_0', '0.25 AIE_2_0.33_0.67_0', '0.5 ADE_0.33_0.67_0', '0.5 AIE_1_0.33_0.67_0', '0.5 AIE_2_0.33_0.67_0', '0.75 ADE_0.33_0.67_0', '0.75 AIE_1_0.33_0.67_0', '0.75 AIE_2_0.33_0.67_0', '0.25 ADE_0.5_0.5_0', '0.25 AIE_1_0.5_0.5_0', '0.25 AIE_2_0.5_0.5_0', '0.5 ADE_0.5_0.5_0', '0.5 AIE_1_0.5_0.5_0', '0.5 AIE_2_0.5_0.5_0', '0.75 ADE_0.5_0.5_0', '0.75 AIE_1_0.5_0.5_0', '0.75 AIE_2_0.5_0.5_0', '0.25 ADE_0.67_0.33_0', '0.25 AIE_1_0.67_0.33_0', '0.25 AIE_2_0.67_0.33_0', '0.5 ADE_0.67_0.33_0', '0.5 AIE_1_0.67_0.33_0', '0.5 AIE_2_0.67_0.33_0', '0.75 ADE_0.67_0.33_0', '0.75 AIE_1_0.67_0.33_0', '0.75 AIE_2_0.67_0.33_0', '0.25 ADE_0.33_0_0.67', '0.25 AIE_1_0.33_0_0.67', '0.25 AIE_2_0.33_0_0.67', '0.5 ADE_0.33_0_0.67', '0.5 AIE_1_0.33_0_0.67', '0.5 AIE_2_0.33_0_0.67', '0.75 ADE_0.

In [106]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

,0.25 ADE_1_0_0,0.25 AIE_1_1_0_0,0.25 AIE_2_1_0_0,0.5 ADE_1_0_0,0.5 AIE_1_1_0_0,0.5 AIE_2_1_0_0,0.75 ADE_1_0_0,0.75 AIE_1_1_0_0,0.75 AIE_2_1_0_0,0.25 ADE_0.33_0.67_0,...,0.75 AIE_2_0.33_0.33_0.33,0.25 ADE_0.33_0.11_0.56,0.25 AIE_1_0.33_0.11_0.56,0.25 AIE_2_0.33_0.11_0.56,0.5 ADE_0.33_0.11_0.56,0.5 AIE_1_0.33_0.11_0.56,0.5 AIE_2_0.33_0.11_0.56,0.75 ADE_0.33_0.11_0.56,0.75 AIE_1_0.33_0.11_0.56,0.75 AIE_2_0.33_0.11_0.56
0,0.981879,-0.559693,-466.296377,0.927689,-1.921774,-1062.384304,1.059205,-2.856251,-1552.073820,0.294000,...,-50.293595,0.325502,-0.280241,0.514463,0.354517,1.230827,31.246367,0.343569,1.127049,103.509811
1,1.009262,-2.233503,-416.450313,0.923860,-0.132557,-1057.284194,0.960908,-0.598231,-1475.486524,0.212046,...,-55.023915,0.397297,-0.296604,-0.691426,0.268737,-0.886476,-11.445713,0.231935,0.901068,-128.620029
2,0.975920,1.015058,-579.014215,0.974418,-2.974549,-933.623270,1.047941,-1.760681,-1547.875260,0.324789,...,-96.611644,0.405380,-0.300463,-23.407121,0.277053,1.104817,11.973783,0.283842,1.058299,-76.590661
3,1.033660,0.228684,-506.245045,0.931048,-0.431420,-976.538338,1.003343,0.603360,-1547.099996,0.321702,...,-50.313661,0.379001,-0.328752,11.537655,0.338940,1.135979,-46.505428,0.337900,-2.587265,144.817807
4,0.956027,-2.262199,-389.870195,0.896394,0.587357,-1094.118018,0.973084,0.492966,-1447.933390,0.419198,...,-52.638416,0.313705,-0.202997,-33.401679,0.424711,-0.926028,61.490814,0.396325,-2.640671,39.181638


In [109]:
df_r.mean()

0.25 ADE_1_0_0                 1.003894
0.25 AIE_1_1_0_0              -0.491697
0.25 AIE_2_1_0_0            -505.664868
0.5 ADE_1_0_0                  1.002046
0.5 AIE_1_1_0_0               -1.028137
                                ...    
0.5 AIE_1_0.33_0.11_0.56       0.115680
0.5 AIE_2_0.33_0.11_0.56       0.869538
0.75 ADE_0.33_0.11_0.56        0.329086
0.75 AIE_1_0.33_0.11_0.56      0.116457
0.75 AIE_2_0.33_0.11_0.56     -3.117905
Length: 90, dtype: float64

In [111]:
df_r.to_csv('/content/drive/Shareddrives/STATS 209 /stats 209 rawr/12_9_23_Equation_1_bio_celegans.csv')